In [1]:
import csv
import os
import json
# Initialize a viewer with the base segmentation for the box of interest
import neuroglancer
import numpy as np
import pandas as pd

In [2]:
def load_next_element(viewer,neurites):
    element = int(neurites[0])
    
    # Define color mapping for the segment(s)
    color_value = "#ef8200"
    segment_color_mapping = {element: color_value}
    
    # Update the viewer
    with viewer.txn() as s:
        # Assign segments using `set()`, since `.segments` is a Neuroglancer `VisibleSegments` object
        s.layers['segmentation'].segments = set([element])
        s.layers['segmentation'].segment_colors = segment_color_mapping
        s.layers['segmentation'].visible = True
    
    print(f'this element is {element}')
    print('Remaining neurites '+ str(len(neurites)))
    
    return element

def query_save_and_reset_viewer(viewer,element,proofread_objects,filename_proofread_list,filename_ref_list):
    ##################################################################################################
    # SAVE
    
    segments = [int(s) for s in list((viewer.state.layers['segmentation'].segments))]
    
    if str(element) in list(proofread_objects.keys()):
        print('ERROR! element already used as a key in the json dict')
        # Grab the reference list segments (all segments in the box >1000 voxels)
        with open(filename_ref_list, 'r') as csv_file:
            reader = csv.reader(csv_file)
            ref_list = [int(value) for row in reader for value in row]  # Flattened list of uint64 values
            proofread_list = set([num for k,v in proofread_objects.items() for num in v])
            neurites = list(set(ref_list).difference(proofread_list))
        return proofread_objects, neurites
        
    elif str(element) not in list(proofread_objects.keys()):
        print('this element does not exist -- updating proofread_elements and reseting viewer')
        proofread_objects[element] = segments # key need s to be element, because element is not always segments[0] and then end up with duplicate keys in split_corrected
           
    with open(filename_proofread_list, 'w') as outfile:
        # json.dump([{k:v} for k,v in proofread_objects.items()], outfile) #use this if want a list of dictionaries rather than a dictionary of all
        json.dump(proofread_objects, outfile)
    
    ################################################################################################
    # RESET 
    # Grab the reference list segments (all segments in the box >1000 voxels)
    with open(filename_ref_list, 'r') as csv_file:
        reader = csv.reader(csv_file)
        ref_list = [int(value) for row in reader for value in row]  # Flattened list of uint64 values
    
    # Grab the split-corrected segments so far
    if os.path.exists(filename_proofread_list):
        with open(filename_proofread_list, 'r') as f:
            proofread_objects = json.load(f)
    else:
        with open(filename_proofread_list, 'w') as outfile:  
            json.dump([], outfile)
        proofread_objects = {}
    
    # Extract proofread segment IDs (make it a flat list) and subtract it from the ref_list to get the remaining list
    proofread_list = set([num for k,v in proofread_objects.items() for num in v])
    neurites = list(set(ref_list).difference(proofread_list))
    ntotal = len(neurites)
    print('Remaining neurites '+ str(len(neurites)))
    
    # Remove all segments from the "segmentation" layer
    with viewer.txn() as txn:
        layer = txn.layers['segmentation']
        layer.segments = set()  # Assign an empty set to clear the segments
    
    with viewer.txn(overwrite=True) as s:
        s.layers['Base Segment Merger'] = neuroglancer.AnnotationLayer()
        s.layers['Base Segment Merger'].filterBySegmentation = ["segments"]
        s.layers['Base Segment Merger'].linkedSegmentationLayer = {"segments": 'segmentation'}
        s.layers['Base Segment Merger'].annotationColor = '#ffa500'
        s.layers['Base Segment Merger'].tool = "annotatePoint"
        
    
    point_type = 'ends'
    with viewer.txn(overwrite=True) as s:
        s.layers[point_type] = neuroglancer.AnnotationLayer()
        s.layers[point_type].annotationColor = '#ffff00'
        s.layers[point_type].tool = "annotatePoint"
        s.layers[point_type].tab = 'Annotations'
    
    # RUN STEP1 again
    
    return proofread_objects, neurites

def save_and_reset_viewer(viewer,proofread_objects,filename_proofread_list,filename_ref_list):
    ##################################################################################################
    # SAVE

    with open(filename_proofread_list, 'w') as outfile:
        # json.dump([{k:v} for k,v in proofread_objects.items()], outfile) #use this if want a list of dictionaries rather than a dictionary of all
        json.dump(proofread_objects, outfile)
    
    ################################################################################################
    # RESET 
    # Grab the reference list segments (all segments in the box >1000 voxels)
    with open(filename_ref_list, 'r') as csv_file:
        reader = csv.reader(csv_file)
        ref_list = [int(value) for row in reader for value in row]  # Flattened list of uint64 values
    
    # Grab the split-corrected segments so far
    if os.path.exists(filename_proofread_list):
        with open(filename_proofread_list, 'r') as f:
            proofread_objects = json.load(f)
    else:
        with open(filename_proofread_list, 'w') as outfile:  
            json.dump([], outfile)
        proofread_objects = {}
    
    # Extract proofread segment IDs (make it a flat list) and subtract it from the ref_list to get the remaining list
    proofread_list = set([num for k,v in proofread_objects.items() for num in v])
    neurites = list(set(ref_list).difference(proofread_list))
    ntotal = len(neurites)
    print('Remaining neurites '+ str(len(neurites)))
    
    # Remove all segments from the "segmentation" layer
    with viewer.txn() as txn:
        layer = txn.layers['segmentation']
        layer.segments = set()  # Assign an empty set to clear the segments
    
    with viewer.txn(overwrite=True) as s:
        s.layers['Base Segment Merger'] = neuroglancer.AnnotationLayer()
        s.layers['Base Segment Merger'].filterBySegmentation = ["segments"]
        s.layers['Base Segment Merger'].linkedSegmentationLayer = {"segments": 'segmentation'}
        s.layers['Base Segment Merger'].annotationColor = '#ffa500'
        s.layers['Base Segment Merger'].tool = "annotatePoint"
        
    
    point_type = 'ends'
    with viewer.txn(overwrite=True) as s:
        s.layers[point_type] = neuroglancer.AnnotationLayer()
        s.layers[point_type].annotationColor = '#ffff00'
        s.layers[point_type].tool = "annotatePoint"
        s.layers[point_type].tab = 'Annotations'
    
    # RUN STEP1 again
    
    return proofread_objects, neurites
    
def check_against_proofread(segments,proofread_objects):
    overlap = []
    num_dup = []
    for k,v in proofread_objects.items():
        overlap.append(len(set(self_segments)&set(v))/len(set(v)))
        num_dup.append(len(set(self_segments)&set(v)))
    
    df = pd.DataFrame({
        "dups": list(proofread_objects.keys()),
        "overlap-percent": overlap,
        "number_seg_lap": num_dup
        }).replace(0, np.nan, inplace=False).dropna()

    return df



In [3]:
# INITIALIZE #
dirpath = '/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/MolecularLayerModel_Michal/Efish_proofreading_box1/'
filename_ref_list = dirpath + "efish_box1_segments_list.csv"
# filename_proofread_list = 'box1_split_corrected.json'
filename_proofread_list = dirpath + 'box1_split_corrected_dict.json'

# Grab the reference list segments (all segments in the box >1000 voxels)
with open(filename_ref_list, 'r') as csv_file:
    reader = csv.reader(csv_file)
    ref_list = [int(value) for row in reader for value in row]  # Flattened list of uint64 values

# Grab the split-corrected segments so far
if os.path.exists(filename_proofread_list):
    with open(filename_proofread_list, 'r') as f:
        proofread_objects = json.load(f)
else:
    with open(filename_proofread_list, 'w') as outfile:  
        json.dump([], outfile)
    proofread_objects = []

# Extract proofread segment IDs (make it a flat list) and subtract it from the ref_list to get the remaining list
proofread_list = set([num for k,v in proofread_objects.items() for num in v])
remaining_list = list(set(ref_list).difference(proofread_list))
neurites = remaining_list
ntotal = len(neurites)
print('Remaining neurites '+ str(len(neurites)))

Remaining neurites 350


In [4]:
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.layers['image'] = neuroglancer.ImageLayer(source='brainmaps://10393113184:ell:roi450um_xyz')
    s.layers['segmentation'] = neuroglancer.SegmentationLayer(
        source='brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930', 
        selected_alpha=0.5
    )

# Initialize a neurites counter
viewer

http://127.0.0.1:53629/v/db8cd997fc34f7c4b4785424f2498c13d2e97d88/

{
  "type": "annotation",
  "source": {
    "url": "local://annotations",
    "transform": {
      "outputDimensions": {
        "x": [
          1.6e-8,
          "m"
        ],
        "y": [
          1.6e-8,
          "m"
        ],
        "z": [
          3e-8,
          "m"
        ]
      }
    }
  },
  "tool": "annotateBoundingBox",
  "tab": "annotations",
  "annotations": [
    {
      "pointA": [
        15075,
        10620,
        1618
      ],
      "pointB": [
        15700,
        11245,
        1951
      ],
      "type": "axis_aligned_bounding_box",
      "id": "8470042ac60231441cf2de9988aabfc42cfe1965"
    }
  ],
  "name": "box1"
}


In [5]:
with viewer.txn(overwrite=True) as s:

    s.layers['Base Segment Merger'] = neuroglancer.AnnotationLayer()
    s.layers['Base Segment Merger'].filterBySegmentation = ["segments"]
    s.layers['Base Segment Merger'].linkedSegmentationLayer = {"segments": 'segmentation'}
    s.layers['Base Segment Merger'].annotationColor = '#ffa500'
    s.layers['Base Segment Merger'].tool = "annotatePoint"
    

point_type = 'ends'
with viewer.txn(overwrite=True) as s:
    s.layers[point_type] = neuroglancer.AnnotationLayer()
    s.layers[point_type].annotationColor = '#ffff00'
    s.layers[point_type].tool = "annotatePoint"
    s.layers[point_type].tab = 'Annotations'


# STEP 1

In [38]:
# Load the first element of the neurites list and make sure it is an int
element = load_next_element(viewer,neurites)

this element is 286782393
Remaining neurites 334


In [36]:
# check to see if duplicate with any already read proofread objects
self_segments = [int(s) for s in list((viewer.state.layers['segmentation'].segments))]
df_dups = check_against_proofread(self_segments,proofread_objects)
display(df_dups)

,dups,overlap-percent,number_seg_lap


# STEP 2

In [37]:
proofread_objects, neurites = query_save_and_reset_viewer(viewer,element,proofread_objects,filename_proofread_list,filename_ref_list)

this element does not exist -- updating proofread_elements and reseting viewer
Remaining neurites 334


# consolidate with existing duplicate

## if you are sure

In [26]:
dupid = '286781626'
consolidated_list = set(proofread_objects[dupid] + self_segments)

In [27]:
proofread_objects[dupid] = list(consolidated_list)

In [28]:
proofread_objects, neurites = save_and_reset_viewer(viewer,proofread_objects,filename_proofread_list,filename_ref_list)

Remaining neurites 342


## if you want to check visually first

In [24]:
# Update the viewer
lname = 'self'
segment_color_mapping = {key: "#ef8200" for key in self_segments}
with viewer2.txn() as s:
    s.layers[lname] = neuroglancer.SegmentationLayer(
        source='brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930', 
        selected_alpha=0.5)
    # Assign segments using `set()`, since `.segments` is a Neuroglancer `VisibleSegments` object
    s.layers[lname].segments = set(self_segments)
    s.layers[lname].segment_colors = segment_color_mapping
    s.layers[lname].visible = True 

In [25]:
        
for d in df_dups['dups'].values:
    segs = list(set(proofread_objects[d]))
    # color_value = generate_random_hex_color()# "#ef8200"
    # segment_color_mapping = {key: generate_random_hex_color() for key in segs}
    
    
    # Update the viewer
    lname = d
    with viewer2.txn() as s:
        s.layers[lname] = neuroglancer.SegmentationLayer(
            source='brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930', 
            selected_alpha=0.5)
        # Assign segments using `set()`, since `.segments` is a Neuroglancer `VisibleSegments` object
        s.layers[lname].segments = set(segs)
        # s.layers[lname].segment_colors = segment_color_mapping
        s.layers[lname].visible = True

In [103]:
dupid = '286797753'
consolidated_list = set(proofread_objects[dupid] + self_segments)

In [104]:
lname = 'consolidated'
segment_color_mapping = {key: "#ef8200" for key in self_segments}
with viewer2.txn() as s:
    s.layers[lname] = neuroglancer.SegmentationLayer(
        source='brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930', 
        selected_alpha=0.5)
    # Assign segments using `set()`, since `.segments` is a Neuroglancer `VisibleSegments` object
    s.layers[lname].segments = set(consolidated_list)
    # s.layers[lname].segment_colors = segment_color_mapping
    s.layers[lname].visible = True 

In [422]:
proofread_objects[dupid] = list(consolidated_list)

In [423]:
proofread_objects, neurites = save_and_reset_viewer(viewer,proofread_objects,filename_proofread_list,filename_ref_list)

Remaining neurites 1909


# check for duplicates

In [185]:
with open(filename_proofread_list, 'r') as f:
    proofread_objects = json.load(f)

In [186]:
def check_duplicates(base_segments):
    '''
    base_segments is a dictionary of all segments that this script checks among
    '''
    df_all = pd.DataFrame()
    dups_list = []
    for self_k,this_cell in base_segments.items():
        # print(dups_list)
        if self_k not in dups_list:
            overlap = []
            num_dup = []
            for x in base_segments.keys():
                overlap.append(len(set(this_cell)&set(base_segments[x]))/len(set(base_segments[x])))
                num_dup.append(len(set(this_cell)&set(base_segments[x])))
    
            df = pd.DataFrame({
                "self": self_k,
                "dups": list(base_segments.keys()),
                "overlap-percent": overlap,
                "number_seg_lap": num_dup
                }).replace(0, np.nan, inplace=False).dropna()
            df = df[df['dups'] != self_k]
            
            if not df.empty:
                dups_list.extend([v for v in df['dups'].values])
                df_all = pd.concat([df_all,df]) 
    return df_all

In [14]:
viewer2 = neuroglancer.Viewer()
with viewer2.txn() as s:
    s.layers['image'] = neuroglancer.ImageLayer(source='brainmaps://10393113184:ell:roi450um_xyz')

# Initialize a neurites counter
viewer2

http://127.0.0.1:53629/v/325ba2e64258d30f78e498f879387d41c5654dc7/

In [234]:
dup_dict = check_duplicates(proofread_objects)

In [235]:
dup_dict[dup_dict['number_seg_lap']>1]

,self,dups,overlap-percent,number_seg_lap
702,286796800,286796414,0.222222,2.0
596,371507331,371508228,0.400000,2.0
412,372667908,372654163,0.200000,2.0
25,285652483,285637634,0.004329,2.0
741,286780872,286796588,0.500000,2.0
935,371524290,286798108,0.750000,6.0
25,371507421,285637634,0.004329,2.0
166,286781584,371507724,0.012195,2.0


In [2325]:
# proofread_objects.pop('371523785')

In [237]:
self_id = '371524290'
dup_list = list(dup_dict[(dup_dict['self']==self_id) & (dup_dict['number_seg_lap']>1)]['dups'].values)

for d in [self_id] + dup_list:
    # Define color mapping for the segment(s)
    segs = list(set(proofread_objects[d]))
    # color_value = generate_random_hex_color()# "#ef8200"
    # segment_color_mapping = {key: color_value for key in segs}


    # Update the viewer
    lname = d
    with viewer2.txn() as s:
        s.layers[lname] = neuroglancer.SegmentationLayer(
            source='brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930', 
            selected_alpha=0.5)
        # Assign segments using `set()`, since `.segments` is a Neuroglancer `VisibleSegments` object
        s.layers[lname].segments = segs
        # s.layers[lname].segment_colors = segment_color_mapping
        s.layers[lname].visible = True
    
    # print(f'this element is {element}')
    # print('Remaining neurites '+ str(len(neurites)))

In [238]:
dup_dict_consolidated = {}

v_list = []
for d in [self_id] + dup_list:
    v_list.append(proofread_objects[d])
dup_dict_consolidated[self_id] = list(set([num for ni in v_list for num in ni]))


In [239]:
# dup_dict_consolidated

segs = dup_dict_consolidated[self_id]
# color_value = generate_random_hex_color()# "#ef8200"
# segment_color_mapping = {key: color_value for key in segs}


# Update the viewer
lname = f'{self_id}_consolidated'
with viewer2.txn() as s:
    s.layers[lname] = neuroglancer.SegmentationLayer(
        source='brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930', 
        selected_alpha=0.5)
    # Assign segments using `set()`, since `.segments` is a Neuroglancer `VisibleSegments` object
    s.layers[lname].segments = set(segs)
    # s.layers[lname].segment_colors = segment_color_mapping
    s.layers[lname].visible = True

In [215]:
dup_list 

['286781112']

In [240]:
# Remove keys using dictionary comprehension
proofread_objects = {k: v for k, v in proofread_objects.items() if k not in dup_list}

In [241]:
proofread_objects[self_id] = dup_dict_consolidated[self_id]

In [242]:
# dup_dict_consolidated

segs = proofread_objects[self_id]
# color_value = generate_random_hex_color()# "#ef8200"
# segment_color_mapping = {key: color_value for key in segs}


# Update the viewer
lname = f'{self_id}_consolidated_final'
with viewer2.txn() as s:
    s.layers[lname] = neuroglancer.SegmentationLayer(
        source='brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930', 
        selected_alpha=0.5)
    # Assign segments using `set()`, since `.segments` is a Neuroglancer `VisibleSegments` object
    s.layers[lname].segments = set(segs)
    # s.layers[lname].segment_colors = segment_color_mapping
    s.layers[lname].visible = True

save proofread_objects if corrected now

In [243]:
with open(filename_proofread_list, 'w') as outfile:
    # json.dump([{k:v} for k,v in proofread_objects.items()], outfile) #use this if want a list of dictionaries rather than a dictionary of all
    json.dump(proofread_objects, outfile)

# combine dicts with same keys (or re-key them)

only duplicates where overlap is not in the base segment merge list

In [20]:
from collections import Counter

import random

def generate_random_hex_color():
  """Generates a random hex color string."""
  hex_color = '#' + ''.join([random.choice('0123456789abcdef') for j in range(6)])
  return hex_color

In [1025]:
# Example usage
random_color = generate_random_hex_color()
print(random_color)

#2363ea


In [1039]:
with open(filename_proofread_list, 'r') as f:
    proofread_objects = json.load(f)
    # proofread_objects = {k: v for l in proofread_objects for k, v in l.items()}

In [1040]:
key_list = [k for l in proofread_objects for k, v in l.items()]

In [1041]:
duplicates = [item for item, count in Counter(key_list).items() if count > 1]

In [1042]:
duplicates

['371523882', '285637634', '372669442', '371507724', '371523648']

In [1043]:
dup_dict = {}
for d in duplicates:
    v_list = []
    for i,l in enumerate(proofread_objects):
        k = list(l.keys())[0]
        if k == d:
            v_list.append(l[k])
    dup_dict[d] = [num for ni in v_list for num in ni]


In [1009]:
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.layers['image'] = neuroglancer.ImageLayer(source='brainmaps://10393113184:ell:roi450um_xyz')
    s.layers['segmentation'] = neuroglancer.SegmentationLayer(
        source='brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930', 
        selected_alpha=0.5
    )

# Initialize a neurites counter
viewer

http://127.0.0.1:58455/v/9d16af3d4b925d1014b2a86b150b63f39a39731e/

In [1036]:
element = list(dup_dict.keys())[5]
segs = [int(v) for v in dup_dict[element]]

In [1037]:

# Define color mapping for the segment(s)
color_value = "#ef8200"
segment_color_mapping = {key: generate_random_hex_color() for key in segs}

# Update the viewer
with viewer.txn() as s:
    # Assign segments using `set()`, since `.segments` is a Neuroglancer `VisibleSegments` object
    s.layers['segmentation'].segments = set(segs)
    s.layers['segmentation'].segment_colors = segment_color_mapping
    s.layers['segmentation'].visible = True

print(f'this element is {element}')
print('Remaining neurites '+ str(len(neurites)))

this element is 371524256
Remaining neurites 3579


In [1053]:
proofread_list_og = [item for dictionary in proofread_objects for sublist in dictionary.values() for item in sublist]
proofread_list_kp = set([num for k,v in proofread_objects_dict.items() for num in v])

In [1054]:
set(proofread_list_og).difference(set(proofread_list_kp))

set()

In [1048]:
# get dictionary of proofread objects from json list
proofread_objects_dict = {k: v for l in proofread_objects for k, v in l.items()}


In [1052]:
# for each duplicate key, replace values with full set from combined dup_dict
for k,v in dup_dict.items():
    proofread_objects_dict[k]=v

In [995]:
len(set([k for l in proofread_objects for k, v in l.items()]))

384

# STEP 1 original

In [ ]:
# INITIALIZE #

filename_ref_list = "efish_box1_segments_list.csv"
filename_proofread_list = 'box1_split_corrected.json'

# Grab the reference list segments (all segments in the box >1000 voxels)
with open(filename_ref_list, 'r') as csv_file:
    reader = csv.reader(csv_file)
    ref_list = [int(value) for row in reader for value in row]  # Flattened list of uint64 values

# Grab the split-corrected segments so far
if os.path.exists(filename_proofread_list):
    with open(filename_proofread_list, 'r') as f:
        proofread_objects = json.load(f)
else:
    with open(filename_proofread_list, 'w') as outfile:  
        json.dump([], outfile)
    proofread_objects = []

# Extract proofread segment IDs (make it a flat list) and subtract it from the remaining list
proofread_list = [item for dictionary in proofread_objects for sublist in dictionary.values() for item in sublist]
remaining_list = [seg_id for seg_id in ref_list if seg_id not in proofread_list]
neurites = remaining_list
ntotal = len(neurites)
print('Remaining neurites '+ str(len(neurites)))

viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.layers['image'] = neuroglancer.ImageLayer(source='brainmaps://10393113184:ell:roi450um_xyz')
    s.layers['segmentation'] = neuroglancer.SegmentationLayer(
        source='brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930', 
        selected_alpha=0.5
    )

# Initialize a neurites counter
viewer

# STEP2 original 

In [933]:
##################################################################################################
# SAVE
# print the split-corrected segments
segments = list((viewer.state.layers['segmentation'].segments))
for i in range( len(segments)):
    segments[i] = int(segments[i])
       

dict_seg = {str(segments[0]): segments}

# save the selected segments
proofread_objects.append(dict_seg)
with open(filename_proofread_list, 'w') as outfile:  
        json.dump(proofread_objects, outfile)

################################################################################################
# RESET 
# Grab the reference list segments (all segments in the box >1000 voxels)
with open(filename_ref_list, 'r') as csv_file:
    reader = csv.reader(csv_file)
    ref_list = [int(value) for row in reader for value in row]  # Flattened list of uint64 values

# Grab the split-corrected segments so far
if os.path.exists(filename_proofread_list):
    with open(filename_proofread_list, 'r') as f:
        proofread_objects = json.load(f)
else:
    with open(filename_proofread_list, 'w') as outfile:  
        json.dump([], outfile)
    proofread_objects = []

# Extract proofread segment IDs (make it a flat list) and subtract it from the remaining list
proofread_list = [item for dictionary in proofread_objects for sublist in dictionary.values() for item in sublist]
remaining_list = [seg_id for seg_id in ref_list if seg_id not in proofread_list]
neurites = remaining_list
ntotal = len(neurites)
print('Remaining neurites '+ str(len(neurites)))

# Remove all segments from the "segmentation" layer
with viewer.txn() as txn:
    layer = txn.layers['segmentation']
    layer.segments = set()  # Assign an empty set to clear the segments

with viewer.txn(overwrite=True) as s:
    s.layers['Base Segment Merger'] = neuroglancer.AnnotationLayer()
    s.layers['Base Segment Merger'].filterBySegmentation = ["segments"]
    s.layers['Base Segment Merger'].linkedSegmentationLayer = {"segments": 'segmentation'}
    s.layers['Base Segment Merger'].annotationColor = '#ffa500'
    s.layers['Base Segment Merger'].tool = "annotatePoint"
    

point_type = 'ends'
with viewer.txn(overwrite=True) as s:
    s.layers[point_type] = neuroglancer.AnnotationLayer()
    s.layers[point_type].annotationColor = '#ffff00'
    s.layers[point_type].tool = "annotatePoint"
    s.layers[point_type].tab = 'Annotations'

# RUN STEP1 again
    

Remaining neurites 3579


# other


In [37]:
vx_sizes = [16,16,30]
t = 'Base Segment Merger'
this_type_points = []
for x in viewer.state.layers[t].annotations:
    if x.segments == None:
        c = [int(y) for y in x.point]
        print(f'Error, no segment for point {c}, for point layer {t}, correct and re-save')
    
    else:
        co_ords = [float(x) for x in list(x.point)]
        co_ords_and_id = ([co_ords[x]*vx_sizes[x] for x in range(3)])
        co_ords_and_id.append(str(x.segments[0][0]))
        
        this_type_points.append((co_ords_and_id))


In [42]:
df = pd.DataFrame(this_type_points,columns = ['x','y','z','segID'])
df.to_csv('Base_Segment_Merger.csv')

In [48]:
df = pd.read_csv('Base_Segment_Merger.csv')
base_seg_merge_points = df[['x','y','z','segID']].values

In [51]:
for pos,point in enumerate(base_seg_merge_points):
    point_array = np.asarray([int(point[x]/vx_sizes[x]) for x in range(3)])
    pa = neuroglancer.PointAnnotation(id=f'bm_{pos}', point = point_array, segments=[[point[3]]])
    s.layers['Base Segment Merger'].annotations.append(pa)        

In [144]:
with open(filename_proofread_list, 'r') as f:
    proofread_objects = json.load(f)

In [145]:
split_dict = {}
for i in proofread_objects:
    for k,v in i.items():
        split_dict[k]=v

In [147]:
proofread_glia_segs_0 = set(split_dict['286796800'])

In [151]:
proofread_glia_segs_1 = set(segments)

In [155]:
segments = list(proofread_glia_segs_1.union(proofread_glia_segs_0))